# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('consulting').getOrCreate()

### Importing the csv file and doing a firt analysis

In [0]:
data = spark.read.csv('/FileStore/tables/cruise_ship_info.csv', header=True, inferSchema=True)

In [0]:
data.columns

Out[3]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [0]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [0]:
data.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows



Check for the presence of null values

In [0]:
from pyspark.sql.functions import col, isnan, when, count
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]
   ).show()

+---------+-----------+---+-------+----------+------+------+-----------------+----+
|Ship_name|Cruise_line|Age|Tonnage|passengers|length|cabins|passenger_density|crew|
+---------+-----------+---+-------+----------+------+------+-----------------+----+
|        0|          0|  0|      0|         0|     0|     0|                0|   0|
+---------+-----------+---+-------+----------+------+------+-----------------+----+



#### One Hot Encoding of the Cruise_Line Column
Since this category seems to be important for the determination of the number of crew members, it will be included in the analysis. Given that it is a string, it can't be used as it is and needs to be converted to numerical values.

In [0]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql.types import StringType, StructType, StructField

In [0]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_Indexed")
indexed_data = indexer.fit(data).transform(data)

In [0]:
encoder = OneHotEncoder(inputCol="Cruise_line_Indexed", outputCol="Cruise_line_onehot")
encoded_data = encoder.fit(indexed_data).transform(indexed_data)
encoded_data.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_Indexed|Cruise_line_onehot|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-------------------+------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|   (19,[16],[1.0])|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|               16.0|   (19,[16],[1.0])|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|                1.0|    (19,[1],[1.0])|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|                1.0|    (19,[1],[1.0])|
|    Destiny|   Carnival| 17|           101.353|

#### Vectorizing the features

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
encoded_data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_Indexed: double (nullable = false)
 |-- Cruise_line_onehot: vector (nullable = true)



In [0]:
assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density', 'Cruise_line_onehot'], outputCol='features')
output = assembler.transform(encoded_data)

In [0]:
final_data = output.select('features', 'crew')
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|(25,[0,1,2,3,4,5,...|3.55|
|(25,[0,1,2,3,4,5,...|3.55|
|(25,[0,1,2,3,4,5,...| 6.7|
|(25,[0,1,2,3,4,5,...|19.1|
|(25,[0,1,2,3,4,5,...|10.0|
|(25,[0,1,2,3,4,5,...| 9.2|
|(25,[0,1,2,3,4,5,...| 9.2|
|(25,[0,1,2,3,4,5,...| 9.2|
|(25,[0,1,2,3,4,5,...| 9.2|
|(25,[0,1,2,3,4,5,...|11.5|
|(25,[0,1,2,3,4,5,...|11.6|
|(25,[0,1,2,3,4,5,...| 6.6|
|(25,[0,1,2,3,4,5,...| 9.2|
|(25,[0,1,2,3,4,5,...| 9.2|
|(25,[0,1,2,3,4,5,...| 9.3|
|(25,[0,1,2,3,4,5,...|11.6|
|(25,[0,1,2,3,4,5,...|10.3|
|(25,[0,1,2,3,4,5,...| 9.2|
|(25,[0,1,2,3,4,5,...| 9.3|
|(25,[0,1,2,3,4,5,...| 9.2|
+--------------------+----+
only showing top 20 rows



#### Building the model
The data will be divided in train and test partitions and the regression model will be fit to the train data

In [0]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
lr = LinearRegression(labelCol='crew')
lr_model = lr.fit(train_data)

#### Model evaluation

I'm going to use the mean squared error and the r square as measures of the model performance

In [0]:
test_results = lr_model.evaluate(test_data)
print('MSE =',  test_results.rootMeanSquaredError)
print('r square =', test_results.r2)

MSE = 1.1537971006591412
r square = 0.8982242683421


In [0]:
final_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



The model has a very good fitting to the data, with 89% of variance explained. The mean square error is 1.15, it is also acceptable since the average crew is 7.79, which means a 14% of percentage of error.

#### Reality check
I will check for higly correlated variables that may explain the good performance of the model but imply that the model is actually meaningless (for instance, if number of passengers and number of crew are highly correlated)

In [0]:
from pyspark.sql.functions import corr

In [0]:
for c in indexed_data.columns:
    indexed_data.select(corr('crew', c)).show()

+---------------------+
|corr(crew, Ship_name)|
+---------------------+
|                 null|
+---------------------+

+-----------------------+
|corr(crew, Cruise_line)|
+-----------------------+
|                   null|
+-----------------------+

+-------------------+
|    corr(crew, Age)|
+-------------------+
|-0.5306565039638852|
+-------------------+

+-------------------+
|corr(crew, Tonnage)|
+-------------------+
|  0.927568811544939|
+-------------------+

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

+------------------+
|corr(crew, length)|
+------------------+
|0.8958566271016579|
+------------------+

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

+-----------------------------+
|corr(crew, passenger_density)|
+-----------------------------+
|         -0.15550928421699717|
+-----------------------------+

+----------------+
|corr(cr

The size of the ship, number of passengers and number of cabins are highly correlated and give already a good indication of the crew needed.

#### Observations
Although not shown, the model performs better when the indexed_data is used instead of the encoded_data.